In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [2]:
dataset = pd.read_csv("dataset.txt", delimiter="|", encoding="ISO-8859-1", index_col="AutoID")
dataset.head()

,Date,Year,Month,MediaType,FullText
AutoID,,,,,
1,8/26/2015,2015,8,twitter,3 ways the internet of things will change Bank...
2,8/5/2015,2015,8,twitter,BankB BankB Name downgrades apple stock to neu...
3,8/12/2015,2015,8,twitter,BankB returns to profit on INTERNET/! board2? ...
4,8/5/2015,2015,8,twitter,BankB tells advisers to exit paulson hedge fun...
5,8/12/2015,2015,8,twitter,BankC may plead guilty over foreign exchange p...


In [3]:
def filter_terms(in_str, sub_list=[]):
    if len(sub_list) == 0:
        return in_str

    for pattern in sub_list:
        in_str = re.sub("\\b" + pattern + "\\b", '', in_str)

    return in_str

In [ ]:
# customer service
mask = (dataset["FullText"].apply(lambda x: 1 if re.search("Bank. credit card", x) else 0))
# (dataset["MediaType"] == "facebook") & \
        
#        (dataset["FullText"].apply(lambda x: len(x)) > 1000)


print("Comments that meet criteria: %s" % len(dataset[mask]))

# sub_list = ["ADDRESS", "INTERNET", "Name", 
#             "twit_hndl", "PHONE", "twit_hndl_BankA", 
#             "twit_hndl_BankB", "twit_hndl_BankC", "twit_hndl_BankD"
#             "BankA", "BankB", "BankC", "BankD"]
# sub_list += nltk.corpus.stopwords.words('english')

# for comment in dataset[mask]["FullText"]:
#     print(comment)
# #     print(filter_terms(comment, sub_list))
#     print()

## Feature Creation

In [144]:
# Text Characteristics
dataset["FullTextLen"] = dataset["FullText"].apply(len)

# Mentions Bank
dataset["has_banka"] = dataset["FullText"].apply(lambda x: 1 if re.search("BankA", x) else 0)
dataset["has_bankb"] = dataset["FullText"].apply(lambda x: 1 if re.search("BankB", x) else 0)
dataset["has_bankc"] = dataset["FullText"].apply(lambda x: 1 if re.search("BankC", x) else 0)
dataset["has_bankd"] = dataset["FullText"].apply(lambda x: 1 if re.search("BankD", x) else 0)

# Has Twitter Handles
dataset["has_banka_handle"] = dataset["FullText"].apply(lambda x: 1 if re.search("twit_hndl_BankA", x) else 0)
dataset["has_bankb_handle"] = dataset["FullText"].apply(lambda x: 1 if re.search("twit_hndl_BankB", x) else 0)
dataset["has_bankc_handle"] = dataset["FullText"].apply(lambda x: 1 if re.search("twit_hndl_BankC", x) else 0)
dataset["has_bankd_handle"] = dataset["FullText"].apply(lambda x: 1 if re.search("twit_hndl_BankD", x) else 0)

# Has components
dataset["has_address"] = dataset["FullText"].apply(lambda x: 1 if re.search("ADDRESS", x) else 0)
dataset["has_link"] = dataset["FullText"].apply(lambda x: 1 if re.search("INTERNET", x) else 0)
dataset["has_phone"] = dataset["FullText"].apply(lambda x: 1 if re.search("PHONE", x) else 0)
dataset["has_name"] = dataset["FullText"].apply(lambda x: 1 if re.search("Name", x) else 0)

# Count Punctuations
dataset["count_exclaimation"] = dataset["FullText"].apply(lambda x: len(re.findall("!", x)) if re.findall("!", x) else 0)
dataset["count_link"] = dataset["FullText"].apply(lambda x: len(re.findall("INTNERET", x)) if re.findall("INTERNET", x) else 0)
dataset["count_name"] = dataset["FullText"].apply(lambda x: len(re.findall("Name", x)) if re.findall("Name", x) else 0)

dataset.head()

,Date,Year,Month,MediaType,FullText,FullTextLen,has_banka,has_bankb,has_bankc,has_bankd,...,has_bankc_handle,has_bankd_handle,has_address,has_link,has_phone,has_name,count_punctuation,count_exclaimation,count_link,count_name
AutoID,,,,,,,,,,,,,,,,,,,,,
1,8/26/2015,2015,8,twitter,3 ways the internet of things will change Bank...,100,0,1,1,0,...,0,0,0,0,0,1,0,0,0,1
2,8/5/2015,2015,8,twitter,BankB BankB Name downgrades apple stock to neu...,95,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
3,8/12/2015,2015,8,twitter,BankB returns to profit on INTERNET/! board2? ...,55,0,1,0,0,...,0,0,0,1,0,0,1,1,0,0
4,8/5/2015,2015,8,twitter,BankB tells advisers to exit paulson hedge fun...,126,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
5,8/12/2015,2015,8,twitter,BankC may plead guilty over foreign exchange p...,82,0,0,1,0,...,0,0,0,1,0,0,1,1,0,0


## Sample Dataset

In [145]:
rows = np.random.choice(dataset.index.values, 10000)
sample = dataset.ix[rows]

## DictVectorizer

In [167]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
cols = ["has_banka", "has_bankb", "has_bankc", "has_bankd", "has_banka_handle", "has_bankb_handle",
        "has_bankc_handle", "has_bankd_handle", "has_address", "has_link", "has_phone", "has_name"]
vec = vec.fit_transform(sample[cols].transpose().to_dict().values())

/Users/vijayv/anaconda/lib/python3.4/site-packages/pandas/core/frame.py:711: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  "columns will be omitted.", UserWarning)


In [170]:
from sklearn.cluster import KMeans
num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(vec)

clusters = km.labels_.tolist()

CPU times: user 2.41 s, sys: 16 ms, total: 2.42 s
Wall time: 2.59 s


In [ ]:
print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :20]:
        print(" %s" % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0], end=",")
    print()
    print()
    
print()
print()

In [111]:
from sklearn.externals import joblib

joblib.dump(km, 'cluster_algo1.pkl')

# km = joblib.load('cluster_algo1.pkl')

['cluster_algo1.pkl', 'cluster_algo1.pkl_01.npy', 'cluster_algo1.pkl_02.npy']